In [26]:
from typing import Any, Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate,PromptTemplate,load_prompt
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.prompts.pipeline import PipelinePromptTemplate


prompt = load_prompt("./prompt.yaml")


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)


prompts = [("intro",intro),("example", example),("start",start),]

full_prompt = PipelinePromptTemplate(final_prompt=final,pipeline_prompts=prompts)
full_prompt.format(character="Pirate",
                   example_question="What is your location?",
example_answer="Arrrrg! That is a secret!! Args Arg!",
question="What is your fav food?")

'\n    \n    You are a role playing assistant.\n    And you are impersonating a Pirate\n\n                                     \n    \n    This is an example of how you talk:\n\n    Human: What is your location?\n    You: Arrrrg! That is a secret!! Args Arg!\n\n                              \n    \n    Start now!\n\n    Human: What is your fav food?\n    You:\n\n'

In [4]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
  def parse(self,text):
    items=  text.strip().split(',')
    return list(map(str.strip,items))
  
  

In [5]:
template = ChatPromptTemplate.from_messages([
  ("system","You are a list generating machine. Everyhitng you are asked will be answered with a comma seperated list of max {max_items} in lowercase. Do NOT reply with anything else."),
  ("human", "{question}")
])



In [8]:
# chain = template | chat | CommaOutputParser()

# chain.invoke({
#   "max_items" : 5,
#   "question": "what are the pokemons?"
# })
chef_prompt = ChatPromptTemplate.from_messages([
  ("system","You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients"),
  ("human","I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat 

In [13]:
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it.",
        ),
        ("human", "{recipe}"),
    ]
)


veg_chain = veg_chef_prompt | chat


final_chain = {"recipe" :chef_chain} | veg_chain

final_chain.invoke({"cuisine":"indian"})


Transforming the classic Chicken Tikka Masala into a vegetarian delight is quite straightforward, thanks to the versatility of paneer (Indian cottage cheese) and tofu, both of which are excellent at absorbing the rich flavors of the marinade and sauce. Here's how you can make a Vegetarian Tikka Masala that retains the essence of the traditional recipe.

### Vegetarian Tikka Masala

#### Ingredients:

**For the Vegetarian Marinade:**
- 1 pound (about 450g) paneer or firm tofu, cut into bite-sized pieces
- 1 cup plain yogurt (Greek yogurt works well)
- 1 tablespoon lemon juice
- 2 teaspoons ground cumin
- 1 teaspoon ground cinnamon
- 2 teaspoons cayenne pepper (adjust according to your spice preference)
- 2 teaspoons freshly ground black pepper
- 1 tablespoon minced fresh ginger
- 1 teaspoon salt

**For the Sauce:**
- 1 tablespoon vegetable oil or ghee
- 1 onion, finely chopped
- 2 garlic cloves, minced
- 1 tablespoon ground cumin
- 1 teaspoon turmeric powder
- 1 teaspoon ground coriande

AIMessageChunk(content="Transforming the classic Chicken Tikka Masala into a vegetarian delight is quite straightforward, thanks to the versatility of paneer (Indian cottage cheese) and tofu, both of which are excellent at absorbing the rich flavors of the marinade and sauce. Here's how you can make a Vegetarian Tikka Masala that retains the essence of the traditional recipe.\n\n### Vegetarian Tikka Masala\n\n#### Ingredients:\n\n**For the Vegetarian Marinade:**\n- 1 pound (about 450g) paneer or firm tofu, cut into bite-sized pieces\n- 1 cup plain yogurt (Greek yogurt works well)\n- 1 tablespoon lemon juice\n- 2 teaspoons ground cumin\n- 1 teaspoon ground cinnamon\n- 2 teaspoons cayenne pepper (adjust according to your spice preference)\n- 2 teaspoons freshly ground black pepper\n- 1 tablespoon minced fresh ginger\n- 1 teaspoon salt\n\n**For the Sauce:**\n- 1 tablespoon vegetable oil or ghee\n- 1 onion, finely chopped\n- 2 garlic cloves, minced\n- 1 tablespoon ground cumin\n- 1 teaspoo